In [1]:
import numpy as np
import matplotlib.pyplot as plt
import neuscitk as ntk

dataset = ntk.LabChartDataset('C:\\Users\\jeann\\Desktop\\NEUSCI 302\\NEUSCI 440\\Lab 3 - Roach Flight.mat')

In [ ]:
# Jeremy's moving window max-min code
from scipy.ndimage import maximum_filter1d, minimum_filter1d

def max_min_moving_window(arr, window_size):
    # Ensure window size is an odd number for symmetrical windowing around each element
    if window_size % 2 == 0:
        raise ValueError("window_size must be an odd number")

    # Apply maximum and minimum filters to get the max and min values in each window
    max_values = maximum_filter1d(arr, size=window_size, mode='reflect')
    min_values = minimum_filter1d(arr, size=window_size, mode='reflect')

    # Compute the max-min difference for each window
    return max_values - min_values

# Jeremy's spike detection code that will only take the first peak within a certain window
from scipy.signal import find_peaks
def find_emg_peaks(arr, height, distance):
    peaks, _ = find_peaks(-arr, -height, distance=distance)
    return peaks

In [ ]:
# Mason's code
from scipy.ndimage import maximum_filter1d, minimum_filter1d


fs = int(dataset.fs)

def get_spike_freqs(data, data_start, data_end): 

    # Select channel data and invert for spike detection
    channel = -data[data_start:data_end]
    time = np.arange(len(channel)) / fs
    
    # Set threshold for detection refrac (bufer tiem to prevent double counting)
    threshold_factor = 2                #adjust if too many / not enough spikes detected
    threshold = threshold_factor * np.std(channel)
    refrac_indices = int(0.01 * fs)

    # Detect peaks
    peaks, properties = find_peaks(channel, height=threshold, distance=refrac_indices)
    print(f'Channel {i}: {len(peaks)} spikes detected')

    spike_train = np.zeros_like(channel, dtype=int)
    spike_train[peaks] = 1

    # Get max minus min
    def max_min_moving_window(arr, window_size):
        # Ensure window size is an odd number for symmetrical windowing around each element
        if window_size % 2 == 0:
            raise ValueError("window_size must be an odd number")

        # Apply maximum and minimum filters to get the max and min values in each window
        max_values = maximum_filter1d(arr, size=window_size, mode='reflect')
        min_values = minimum_filter1d(arr, size=window_size, mode='reflect')

        # Compute the max-min difference for each window
        return max_values - min_values

    minmax = max_min_moving_window (channel, int(dataset.fs)+1)

    return peaks, properties, spike_train, minmax

pks, props, spike_train, minmax = get_spike_freqs(dataset.get_block(2)[0], 0, 60*seconds)